In [1]:
import pandas as pd
from requests import get
from bs4 import BeautifulSoup
import os
import path
import re

import unicodedata
import re
import json

import nltk
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.corpus import stopwords



In [ ]:
def find_website(string): 
    url = re.findall(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', string)
    return url 


In [ ]:
# git_repo_list = []

# # Top 10 JS
# url = 'https://github.com/search?l=Shell&q=stars%3A%3E0&s=stars&type=Repositories'
# headers = {'User-Agent': 'Codeup Ada Data Science'}
# response = get(url, headers=headers)
# soup = BeautifulSoup(response.content, 'html.parser')
# blob = soup.find_all('a', class_="v-align-middle")
# git_site = find_website(str(blob))[0::2]
# git_site
# for git in git_site:
#     git_repo_list.append(git)



In [ ]:
# git_ = git_repo_list[91]
# readme_end = '/blob/master/README.md'
# readme_url = git_ + readme_end
# headers = {'User-Agent': 'Codeup Ada Data Science'}
# response = get(readme_url, headers=headers)
# soup = BeautifulSoup(response.content, 'html.parser')
# body = soup.find('article', class_="markdown-body entry-content p-3 p-md-6").text.strip()
# body

    

In [12]:
# Grabs the language and body for each git repo as a dictionary and adds to a list
def get_git_info(star_list):
    
    github_base_url = 'https://github.com'
    git_repo_url = []
    
    for repo in star_list:
        git_repo_url.append(github_base_url + repo)

    list_of_git_info = []

    for repo_address in git_repo_url:
        git_repo_url = repo_address 
        headers = {'User-Agent': 'Codeup Ada Data Science'}
        response = get(git_repo_url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        language = soup.find('span', class_='lang')

#       This adds the part to get to the readme and scrapes the body  
        readme_end = '/blob/master/README.md'
        readme_url = git_repo_url + readme_end
        headers = {'User-Agent': 'Codeup Ada Data Science'}
        response = get(readme_url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        body = soup.find('article', class_="markdown-body entry-content p-3 p-md-6")

#       Combines the language and body to a dictionary, if no language on the repo it is ignored
        if language != None:
            language = str(language)
            list_of_git_info.append({'Language': re.findall(r'>(.*?)<', language)[0],
                                     'Body': body})

    return list_of_git_info


In [14]:
bulk = get_git_info(star_list)

In [45]:
bulk[0]['Body'].text.strip()

"Welcome to freeCodeCamp.org's open source codebase and curriculum!\nfreeCodeCamp.org is a friendly community where you can learn to code for free. It is run by a donor-supported 501(c)(3) nonprofit with the goal of helping millions of busy adults transition into tech. Our community has already helped more than 10,000 people get their first developer job.\nOur full-stack web development curriculum is completely free and self-paced. We have thousands of interactive coding challenges to help you expand your skills.\nTable of Contents\n\nCertifications\nThe Learning Platform\nFound a Bug\nFound a Security Issue\nContributing\nLicense\n\nCertifications\nfreeCodeCamp.org offers several free developer certifications. Each of these certifications involves building 5 required web app projects, along with hundreds of optional coding challenges to help you prepare for those projects. We estimate that each certification will take a beginning programmer around 300 hours to earn.\nEach of these 30 

In [46]:
for git in bulk:
    bulk[git]['Body'].text.strip()

TypeError: list indices must be integers or slices, not dict

In [2]:
# read the html file from a file in the curent working directory ('GitHubTopStars_repos.html')
def read_GitHub_most_stars(filename):
    with open(filename) as f:
        contents = f.read()
    return contents

In [3]:
def parse_urls(contents):
    soup = BeautifulSoup(contents, 'html.parser')
    title = soup.title
    orig_text = soup.text
    stars = soup.find_all('a', class_= 'v-align-middle')
    #  find all the "hrefs", which are the trending titles
    return(re.findall(r'href="(.*?)"', str(stars)))


In [4]:
#  turn off the web access for now and just read the cached files from the working directory
#  contents = get_GitHub_most_stars()
list_of_html_files = ['GitHubTopStars_repos-page1.html','GitHubTopStars_repos-page2.html',\
                      'GitHubTopStars_repos-page3.html','GitHubTopStars_repos-page4.html',\
                      'GitHubTopStars_repos-page5.html','GitHubTopStars_repos-page6.html',\
                      'GitHubTopStars_repos-page7.html','GitHubTopStars_repos-page8.html',\
                      'GitHubTopStars_repos-page9.html','GitHubTopStars_repos-page10.html',\
                      'GitHubTopStars_repos-page11.html','GitHubTopStars_repos-page12.html',\
                      'GitHubTopStars_repos-page13.html']

star_list = []
for html_file in list_of_html_files:
    contents = read_GitHub_most_stars(html_file)
    next_list = parse_urls(contents)
    star_list = star_list + next_list

In [7]:
star_list

['/freeCodeCamp/freeCodeCamp',
 '/996icu/996.ICU',
 '/vuejs/vue',
 '/twbs/bootstrap',
 '/facebook/react',
 '/tensorflow/tensorflow',
 '/EbookFoundation/free-programming-books',
 '/sindresorhus/awesome',
 '/getify/You-Dont-Know-JS',
 '/robbyrussell/oh-my-zsh',
 '/jwasham/coding-interview-university',
 '/microsoft/vscode',
 '/electron/electron',
 '/facebook/create-react-app',
 '/vinta/awesome-python',
 '/flutter/flutter',
 '/donnemartin/system-design-primer',
 '/CyC2018/CS-Notes',
 '/nodejs/node',
 '/FortAwesome/Font-Awesome',
 '/mrdoob/three.js',
 '/ytdl-org/youtube-dl',
 '/microsoft/TypeScript',
 '/webpack/webpack',
 '/atom/atom',
 '/angular/angular',
 '/mui-org/material-ui',
 '/socketio/socket.io',
 '/ant-design/ant-design',
 '/hakimel/reveal.js',
 '/toddmotto/public-apis',
 '/moby/moby',
 '/tensorflow/models',
 '/kubernetes/kubernetes',
 '/laravel/laravel',
 '/jquery/jquery',
 '/mrdoob/three.js',
 '/ytdl-org/youtube-dl',
 '/justjavac/free-programming-books-zh_CN',
 '/GoogleChrome/pup

In [8]:
github_base_url = 'https://github.com'

git_repo_url = []

for repo in star_list:
    git_repo_url.append(github_base_url + repo)
    
git_repo_url   
    

['https://github.com/freeCodeCamp/freeCodeCamp',
 'https://github.com/996icu/996.ICU',
 'https://github.com/vuejs/vue',
 'https://github.com/twbs/bootstrap',
 'https://github.com/facebook/react',
 'https://github.com/tensorflow/tensorflow',
 'https://github.com/EbookFoundation/free-programming-books',
 'https://github.com/sindresorhus/awesome',
 'https://github.com/getify/You-Dont-Know-JS',
 'https://github.com/robbyrussell/oh-my-zsh',
 'https://github.com/jwasham/coding-interview-university',
 'https://github.com/microsoft/vscode',
 'https://github.com/electron/electron',
 'https://github.com/facebook/create-react-app',
 'https://github.com/vinta/awesome-python',
 'https://github.com/flutter/flutter',
 'https://github.com/donnemartin/system-design-primer',
 'https://github.com/CyC2018/CS-Notes',
 'https://github.com/nodejs/node',
 'https://github.com/FortAwesome/Font-Awesome',
 'https://github.com/mrdoob/three.js',
 'https://github.com/ytdl-org/youtube-dl',
 'https://github.com/micros

In [ ]:
https://github.com/freeCodeCamp/freeCodeCamp

In [6]:
def basic_clean(repo):
    repo = re.sub(r'\s', ' ', repo).lower()
    repo = unicodedata.normalize('NFKD', repo)\
        .encode('ascii', 'ignore')\
        .decode('utf-8', 'ignore')
    repo = re.sub(r"[^a-z0-9'\s]", '', repo)
    return repo


def tokenize(repo):
    tokenizer = nltk.tokenize.ToktokTokenizer()
    repo = tokenizer.tokenize(repo, return_str=True)
    return repo
    

def stem(repo):
    ps = nltk.porter.PorterStemmer()
    stems = [ps.stem(word) for word in repo.split()]
    repo_stemmed = ' '.join(stems)
    return repo_stemmed


def lemmatize(repo):
    wnl = nltk.stem.WordNetLemmatizer()
    lemmas = [wnl.lemmatize(word) for word in repo.split()]
    repo_lemmatized = ' '.join(lemmas)
    return repo_lemmatized


def remove_stopwords(repo, extra_words = [], exclude_words = []):
    stopword_list = stopwords.words('english')
    [stopword_list.append(word) for word in extra_words if word not in stopword_list]
    [stopword_list.remove(word) for word in exclude_words if word in stopword_list]
    words = repo.lower().split()
    filtered_words = [w for w in words if w not in stopword_list]
    repo_without_stopwords = ' '.join(filtered_words)
    return repo_without_stopwords


def prep_repo(dictionary_repo, extra_words = [], exclude_words = []):
    cleaned_dict = {
    'language': dictionary_repo['Language'],
    'original': dictionary_repo['Body'],
    'stemmed': stem(dictionary_repo['Body']),
    'lemmatized': lemmatize(dictionary_repo['Body']),
    'clean': remove_stopwords(basic_clean(dictionary_repo['Body']), extra_words, exclude_words),
        }

    return cleaned_dict

def prepare_repo_data(dictionary_repo, extra_words = [], exclude_words = []):
    clean_dict_list = []
    
    for repo in dictionary_repo:
        clean_dict_list.append(prep_repo(repo, extra_words, exclude_words))
        
    return clean_dict_list

def get_git_df(gits):
    data = prepare_repo_data(get_git_info(gits))
    df = pd.DataFrame(data)
    df.set_index('language', inplace=True)
    return df
    